In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


학습데이터 생성

In [2]:
# xData = [공부 시간, 과외 시간]
temp = [[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]]
print(temp)
print(type(temp))

xData = np.array(temp)
print(xData)
print(type(xData))

[[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]]
<class 'list'>
[[ 2  3]
 [ 4  3]
 [ 6  4]
 [ 8  6]
 [10  7]
 [12  8]
 [14  9]]
<class 'numpy.ndarray'>


결과 데이터 생성 -> 실제값

In [3]:
# yData = [합격여부]
temp = [0,0,0,1,1,1,1]
print(temp)
print(type(temp))

yData = np.array(temp)
print(yData)
print(type(yData))

yData = np.array(temp).reshape(7,1)
print(yData)
print(type(yData))

[0, 0, 0, 1, 1, 1, 1]
<class 'list'>
[0 0 0 1 1 1 1]
<class 'numpy.ndarray'>
[[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>


xDate와 yData를 저장할 placeholder 만듦

In [4]:
X = tf.placeholder(dtype=tf.float32, shape=[None, 2])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1])

기울기(가중치, a)와 y절편(바이어스, b)를 임의의 값으로 정함

In [5]:
# random_uniform : 균등분포, random_normal : 정규분포
a = tf.Variable(tf.random_uniform([2,1]), dtype=tf.float32)
b = tf.Variable(tf.random_uniform([1]), dtype=tf.float32)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('a1: {}, a2: {}, b: {}'.format(sess.run(a)[0], sess.run(a)[1], sess.run(b)))
print('{}'.format(sess.run(a)))
print('{}'.format(sess.run(b)))

a1: [0.946555], a2: [0.6024834], b: [0.7647165]
[[0.946555 ]
 [0.6024834]]
[0.7647165]


시그모이드 방정식, 오차 함수, 경사하강법

In [6]:
y = tf.sigmoid(tf.matmul(X, a) + b)
loss = -tf.reduce_mean(Y * tf.log(y) + (1 - Y) * tf.log(1-y))
gradient_descent = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

sigmoid() 메소드를 실행한 예상값 계산

In [11]:
# sigmoid() 메소드의 실행 결과(y)가 0.5 이상이면 1을, 0.5 미만이며 0을 리턴함
sess = tf.Session()
#predicted = tf.cast(tf.constant([1.9, 2.1]), dtype=tf.int32) # 실수 -> 정수
#print(sess.run(predicted)) # [1 2]
#predicted = tf.cast(0.4 >= 0.5, dtype=tf.int32) # boolean : True(1), False(0)
#print(sess.run(predicted)) # 0 

predicted = tf.cast(y >= 0.5, dtype=tf.float32)

예측값과 실제값이 일치하는 정도(정확도) 계산

In [12]:
# reduce_mean : 평균
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.int32))

학습

In [17]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(3001):
    # 기울기, y절편, 오차함수, 경사하강
    a_, b_, loss_, _ = sess.run([a, b, loss, gradient_descent], feed_dict={X: xData, Y: yData})
    if i % 300 == 0:
        print('Epoch(학습횟수): %4d, a1(공부시간기울기): %8.5f, a2(과외시간기울기): %8.5f, b(y절편): %8.5f, loss(오차함수): %8.5f' % (i, a_[0], a_[1], b_, loss_))


Epoch(학습횟수):    0, a1(공부시간기울기):  0.41376, a2(과외시간기울기):  0.29746, b(y절편):  0.62997, loss(오차함수):  1.92117
Epoch(학습횟수):  300, a1(공부시간기울기):  0.85002, a2(과외시간기울기): -0.62544, b(y절편): -2.30587, loss(오차함수):  0.27468
Epoch(학습횟수):  600, a1(공부시간기울기):  0.84536, a2(과외시간기울기): -0.34268, b(y절편): -3.80266, loss(오차함수):  0.19605
Epoch(학습횟수):  900, a1(공부시간기울기):  0.75209, a2(과외시간기울기): -0.00742, b(y절편): -4.88376, loss(오차함수):  0.15276
Epoch(학습횟수): 1200, a1(공부시간기울기):  0.64570, a2(과외시간기울기):  0.30508, b(y절편): -5.73742, loss(오차함수):  0.12472
Epoch(학습횟수): 1500, a1(공부시간기울기):  0.54452, a2(과외시간기울기):  0.58311, b(y절편): -6.44425, loss(오차함수):  0.10509
Epoch(학습횟수): 1800, a1(공부시간기울기):  0.45320, a2(과외시간기울기):  0.82795, b(y절편): -7.04782, loss(오차함수):  0.09063
Epoch(학습횟수): 2100, a1(공부시간기울기):  0.37222, a2(과외시간기울기):  1.04378, b(y절편): -7.57467, loss(오차함수):  0.07959
Epoch(학습횟수): 2400, a1(공부시간기울기):  0.30071, a2(과외시간기울기):  1.23505, b(y절편): -8.04222, loss(오차함수):  0.07090
Epoch(학습횟수): 2700, a1(공부시간기울기):  0.23746, a2(과외시간기울기):  1.40571,

테스트 데이터 만듦

In [34]:
new_x = np.array([6,5]).reshape(1,2)
print(new_x)

# sigmoid() 메소드의 연산 결과에 테스트 데이터 대입 후 연산
result, new_y = sess.run([predicted, y], feed_dict={X:new_x})
print('공부시간: %d, 과외시간: %d' % (new_x[:,0], new_x[:,1]))
print('합격여부: %d, 합격확률: %f' % (result, new_y))

for i in range(11):
    new_x = np.array([0,i]).reshape(1,2)
    result, new_y = sess.run([predicted, y], feed_dict={X:new_x})
    print('공부시간: %d, 과외시간: %2d' % (new_x[:,0], new_x[:,1]), end=' ')
    print('합격여부: %s, 합격확률: %f%%' % (' 합격 ' if result==1 else '불합격', new_y))

[[6 5]]
공부시간: 6, 과외시간: 5
합격여부: 1, 합격확률: 0.509674
공부시간: 0, 과외시간:  0 합격여부: 불합격, 합격확률: 0.000144%
공부시간: 0, 과외시간:  1 합격여부: 불합격, 합격확률: 0.000685%
공부시간: 0, 과외시간:  2 합격여부: 불합격, 합격확률: 0.003249%
공부시간: 0, 과외시간:  3 합격여부: 불합격, 합격확률: 0.015260%
공부시간: 0, 과외시간:  4 합격여부: 불합격, 합격확률: 0.068623%
공부시간: 0, 과외시간:  5 합격여부: 불합격, 합격확률: 0.259426%
공부시간: 0, 과외시간:  6 합격여부:  합격 , 합격확률: 0.624835%
공부시간: 0, 과외시간:  7 합격여부:  합격 , 합격확률: 0.887874%
공부시간: 0, 과외시간:  8 합격여부:  합격 , 합격확률: 0.974126%
공부시간: 0, 과외시간:  9 합격여부:  합격 , 합격확률: 0.994444%
공부시간: 0, 과외시간: 10 합격여부:  합격 , 합격확률: 0.998826%
